### Train NMD efficiency Predictor upon RandomForestRegressor

In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [2]:
# load dataset
df= pd.read_csv('tcga_dataset.csv')
df.shape

(4257, 49)

In [3]:
'''
Description of columns
        downstream_exon_count (as Downstream exon count): The number of exons downstream of the PTC
        last_exon (as Last exon): 1 if the PTC is located on the last exon; 0 otherwise
        PTC_to_start_codon (as Dist PTC to start codon): The distance between the PTC to start codon
        dist_to_stop_codon (as Dist PTC to normal stop codon): The distance between the PTC and normal stop codon
        PTC_exon_length (as PTC-containing exon length): The length of the PTC-containing exon
        PTC_to_intron (as Dist PTC to downstream EJ): The distance between the PTC and downstream exon junction
        upstream_exon_count (as Upstream exon count): The number of exons upstream of the PTC
        mRNA_half_life (as mRNA half-life): The half-life of mRNA
        50nt_to_last_EJ (as Last 50nt penultimate exon): 1 if the PTC is located on the last exon; 0 otherwise
        LOEUF (as LOEUF): Gene-level degree of mutational constraints
        AF: (as Allele frequency): Allele frequency in gnomAD database
        5UTR_length: (as 5'UTR length) : The length of 5'UTR
        3UTR_length: (as 3'UTR length) : The length of 3'UTR
        Transcript_length: (as Transcript length) : The length of the transcript
'''

# we expect to match names of columns to your datasets
cols = ['downstream_exon_count', 'last_exon', 'PTC_to_start_codon',
        'dist_to_stop_codon', 'PTC_exon_length', 'PTC_to_intron','upstream_exon_count',
        'mRNA_half_life','50nt_to_last_EJ','LOEUF', 'AF', '5UTR_length', '3UTR_length', 'Transcript_length']

df[cols].head(1)

,downstream_exon_count,last_exon,PTC_to_start_codon,dist_to_stop_codon,PTC_exon_length,PTC_to_intron,upstream_exon_count,mRNA_half_life,50nt_to_last_EJ,LOEUF,AF,5UTR_length,3UTR_length,Transcript_length
0,1,0,996,369,89,84,7,592.689818,0,0.737,0.0,356,370,1722


In [4]:
df[cols].isnull().sum()

downstream_exon_count      0
last_exon                  0
PTC_to_start_codon         0
dist_to_stop_codon         0
PTC_exon_length            0
PTC_to_intron              0
upstream_exon_count        0
mRNA_half_life             0
50nt_to_last_EJ            0
LOEUF                    130
AF                         0
5UTR_length                0
3UTR_length                0
Transcript_length          0
dtype: int64

In [5]:
df = df.loc[df['LOEUF'].notnull()].reset_index(drop=True)
df.shape

(4127, 49)

In [6]:
regr = RandomForestRegressor(max_features = 3, n_estimators=10000, n_jobs=-1)

In [7]:
# applying cross-valiation method for evaluation
scores = cross_val_score(regr, df[cols], df['NMD_efficiency'], cv=5)
print(f'r2: {np.mean(scores):5f}, std: {np.std(scores):5f}')

r2: 0.437274, std: 0.069419


In [8]:
X = df[cols]
y = df['NMD_efficiency']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)

In [9]:
regr.fit(X_train, y_train)
regr.score(X_test, y_test)
print(f'r2: {regr.score(X_test, y_test):5f}')

r2: 0.459689


### Importance of trained randomforest regressor

In [10]:
importance_df = pd.DataFrame({'feature':regr.feature_names_in_, 
                              'importance': regr.feature_importances_})

importance_df 

,feature,importance
0,downstream_exon_count,0.134166
1,last_exon,0.079576
2,PTC_to_start_codon,0.093868
3,dist_to_stop_codon,0.083519
4,PTC_exon_length,0.090316
5,PTC_to_intron,0.085420
6,upstream_exon_count,0.050611
7,mRNA_half_life,0.060460
8,50nt_to_last_EJ,0.020069
9,LOEUF,0.065284


In [11]:
# Save the trained model
pickle.dump(regr, open('nmd_eff_predictor', 'wb'))